In [22]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import math
import random
from torch.cuda.amp import autocast, GradScaler

In [23]:
nu = 5e-4            
rho = 1
U_inlet = 1.0

cyl_center = (0.5, 0.5)
cyl_radius = 0.05

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [24]:
import torch
import torch.nn as nn
import math

def xavier_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

class PINN(nn.Module):
    def __init__(self, layers=[3] + [64]*10 + [4]):
        """
        Tanh-based PINN for URANS + RNG k-ε model.
        Input: [x, y, t]
        Output: [ψ, p, k, ε]
        """
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 2):
            self.layers.append(nn.Linear(layers[i], layers[i + 1]))
        self.output_layer = nn.Linear(layers[-2], layers[-1])
        
        # Activation
        self.activation = nn.Tanh()

        # Initialize weights 
        self.apply(xavier_init)

    def forward(self, x):
        """Forward pass: returns ψ, p, k, ε"""
        h = x
        for layer in self.layers:
            h = self.activation(layer(h))
        out = self.output_layer(h)
        ψ = out[:, 0:1]
        p = out[:, 1:2] 
        self.softplus = nn.Softplus() 
        k = self.softplus(out[:, 2:3]) + 1e-6 
        eps = self.softplus(out[:, 3:4]) + 1e-6 
        return ψ, p, k, eps

    def velocity_pressure(self, x):
        """Compute velocity (u,v) from streamfunction ψ."""
        x.requires_grad_(True)
        ψ, p, k, eps = self.forward(x)
        grads = torch.autograd.grad(
            ψ, x, grad_outputs=torch.ones_like(ψ),
            create_graph=True,
            retain_graph=True
        )[0]
        ψ_x = grads[:, 0:1]
        ψ_y = grads[:, 1:2]
        u = ψ_y
        v = -ψ_x
        return u, v, p



In [25]:
data_dir = "/kaggle/input/cfd-flow-pass-a-cylinder-0-01"
t_start = 0
t_end = 501
dt = 0.01

xyt_list = []
uvp_list = []

for i in range(t_start, t_end):
    csv_path = os.path.join(data_dir, f"result_{i}.csv")
    df = pd.read_csv(csv_path)

    # timestep
    t_val = i * dt
    t_column = np.full_like(df["Points:0"].values, fill_value=t_val, dtype=np.float32)

    # (x, y, t)
    xyt = np.stack([
        df["Points:0"].values,
        df["Points:1"].values,
        t_column
    ], axis=1)  

    # (u, v, p)
    uvp = np.stack([
        df["u:0"].values,
        df["u:1"].values,
        df["p"].values
    ], axis=1)  

    xyt_list.append(xyt)
    uvp_list.append(uvp)

xyt_tensor = torch.tensor(np.concatenate(xyt_list, axis=0), dtype=torch.float32)
uvp_tensor = torch.tensor(np.concatenate(uvp_list, axis=0), dtype=torch.float32)

print(xyt_tensor.shape)
print(uvp_tensor.shape)

torch.Size([20806530, 3])
torch.Size([20806530, 3])


In [26]:
import torch

# Assuming xyt_tensor is your (N, 3) tensor [x, y, t]
xyt_scaled = xyt_tensor.clone()

# Compute min and max for each column
mins = xyt_tensor.min(dim=0)[0]
maxs = xyt_tensor.max(dim=0)[0]

# Scale each dimension to [-1, 1]
xyt_scaled = 2 * (xyt_tensor - mins) / (maxs - mins) - 1

print("Scaled shape:", xyt_scaled.shape)
print("Min values:", xyt_scaled.min(dim=0)[0])
print("Max values:", xyt_scaled.max(dim=0)[0])


Scaled shape: torch.Size([20806530, 3])
Min values: tensor([-1., -1., -1.])
Max values: tensor([1., 1., 1.])


In [27]:
scaling_params = {
    "mins": mins,
    "maxs": maxs
}

# Example of inverse transform (from scaled → original)
def inverse_scale(x_scaled, mins, maxs):
    return 0.5 * (x_scaled + 1) * (maxs - mins) + mins

In [28]:
t_min, t_max = mins[2], maxs[2]
t0_scaled = 2 * (0.0 - t_min) / (t_max - t_min) - 1
print("Scaled t=0 corresponds to:", t0_scaled.item())


Scaled t=0 corresponds to: -1.0


In [29]:
time_col_scaled = xyt_scaled[:, 2]

# Use tolerance instead of exact equality
mask_ic = torch.isclose(time_col_scaled, torch.tensor(t0_scaled, dtype=time_col_scaled.dtype), atol=1e-6)

xyt_tensor_ic = xyt_scaled[mask_ic]
uvp_tensor_ic = uvp_tensor[mask_ic]

print("IC xyt:", xyt_tensor_ic.shape)
print("IC uvp:", uvp_tensor_ic.shape)


/tmp/ipykernel_39/2071658522.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask_ic = torch.isclose(time_col_scaled, torch.tensor(t0_scaled, dtype=time_col_scaled.dtype), atol=1e-6)


IC xyt: torch.Size([41530, 3])
IC uvp: torch.Size([41530, 3])


In [30]:
def sample_points(inputs,
                  outputs,
                  N_per_timestep,
                  seed=None,
                  device="cuda"):

    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)

    # ⚠️ Adjusted for scaled coordinates [-1, 1]
    # Original domain: x ∈ [0, 2.5], y ∈ [0, 1]
    # After scaling: x_scaled = 2*(x/Lx) - 1, y_scaled = 2*(y/Ly) - 1
    # So:
    #   x=0   → -1
    #   x=2.5 →  1
    #   y=0   → -1
    #   y=1   →  1

    # Convert all physical coordinates into scaled space manually if needed
    # but assuming inputs are already scaled to [-1, 1]:

    # Obstacle (original center (0.5, 0.5), radius 0.05)
    obstacle_center = (
        2 * (0.5 / 2.5) - 1,  # scaled x center
        2 * (0.5 / 1.0) - 1   # scaled y center
    )
    obstacle_radius = 2 * (0.05 / 2.5)  # scaled by x-domain

    # Regions (convert from physical to scaled)
    def scale_x(x): return 2 * (x / 2.5) - 1
    def scale_y(y): return 2 * (y / 1.0) - 1

    refine_1 = ((scale_x(0.35), scale_x(0.65)), (scale_y(0.35), scale_y(0.65)))
    refine_2 = ((scale_x(0.65), scale_x(2.5)), (scale_y(0.35), scale_y(0.65)))

    percent_1 = 40
    percent_2 = 35
    percent_3 = 100 - (percent_1 + percent_2)

    # Convert to numpy
    pts_np = inputs.cpu().numpy()
    uvp_np = outputs.cpu().numpy()

    # Mask out obstacle region
    dx = pts_np[:, 0] - obstacle_center[0]
    dy = pts_np[:, 1] - obstacle_center[1]
    mask = dx**2 + dy**2 >= obstacle_radius**2
    pts_np = pts_np[mask]
    uvp_np = uvp_np[mask]

    # Timesteps
    timesteps = np.unique(pts_np[:, 2])
    timesteps = np.random.choice(timesteps, size=20, replace=False)

    pts_final_list, uvp_final_list = [], []

    for t in timesteps:
        mask_t = np.abs(pts_np[:, 2] - t) < 1e-12
        pts_t, uvp_t = pts_np[mask_t], uvp_np[mask_t]

        # --- region 1
        mask_r1 = (
            (pts_t[:, 0] >= refine_1[0][0]) & (pts_t[:, 0] <= refine_1[0][1]) &
            (pts_t[:, 1] >= refine_1[1][0]) & (pts_t[:, 1] <= refine_1[1][1])
        )
        pts_r1, uvp_r1 = pts_t[mask_r1], uvp_t[mask_r1]

        # --- region 2
        mask_r2 = (
            (pts_t[:, 0] >= refine_2[0][0]) & (pts_t[:, 0] <= refine_2[0][1]) &
            (pts_t[:, 1] >= refine_2[1][0]) & (pts_t[:, 1] <= refine_2[1][1])
        )
        pts_r2, uvp_r2 = pts_t[mask_r2], uvp_t[mask_r2]

        # --- region 3 = rest
        mask_r3 = ~(mask_r1 | mask_r2)
        pts_r3, uvp_r3 = pts_t[mask_r3], uvp_t[mask_r3]

        # How many per region
        N1 = int(N_per_timestep * percent_1 / 100.0)
        N2 = int(N_per_timestep * percent_2 / 100.0)
        N3 = N_per_timestep - N1 - N2

        def sample_region(pts_region, uvp_region, N):
            if len(pts_region) == 0:
                return np.empty((0, 3)), np.empty((0, 3))
            if len(pts_region) < N:
                idx = np.random.choice(len(pts_region), size=N, replace=True)
            else:
                idx = np.random.choice(len(pts_region), size=N, replace=False)
            return pts_region[idx], uvp_region[idx]

        # Sample each region
        sp1, uv1 = sample_region(pts_r1, uvp_r1, N1)
        sp2, uv2 = sample_region(pts_r2, uvp_r2, N2)
        sp3, uv3 = sample_region(pts_r3, uvp_r3, N3)

        pts_final_list.append(np.vstack([sp1, sp2, sp3]))
        uvp_final_list.append(np.vstack([uv1, uv2, uv3]))

    # Concatenate
    pts_final = np.vstack(pts_final_list)
    uvp_final = np.vstack(uvp_final_list)

    # Shuffle
    perm = np.random.permutation(len(pts_final))
    pts_final = pts_final[perm]
    uvp_final = uvp_final[perm]

    # Torch tensors
    points = torch.tensor(pts_final, dtype=torch.float32, device=device)
    uvp = torch.tensor(uvp_final, dtype=torch.float32, device=device)
    u = uvp[:, 0:1]
    v = uvp[:, 1:2]
    p = uvp[:, 2:3]

    return points, u, v, p


In [31]:
import torch
import numpy as np

def sample_collocation_points(N_per_timestep,
                              seed=None,
                              device="cuda"):

    total_time = 5.0
    dt = 0.01
    n_timesteps_per_iter = 20

    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)

    # --- Physical domain ---
    domain_x, domain_y, domain_t = (0.0, 2.5), (0.0, 1.0), (0.0, total_time)

    # --- Scaling functions for [-1, 1] ---
    def scale_x(x): return 2 * (x - domain_x[0]) / (domain_x[1] - domain_x[0]) - 1
    def scale_y(y): return 2 * (y - domain_y[0]) / (domain_y[1] - domain_y[0]) - 1
    def scale_t(t): return 2 * (t - domain_t[0]) / (domain_t[1] - domain_t[0]) - 1

    # --- Scaled domain and regions ---
    domain_x_scaled = (scale_x(domain_x[0]), scale_x(domain_x[1]))  # (-1, 1)
    domain_y_scaled = (scale_y(domain_y[0]), scale_y(domain_y[1]))  # (-1, 1)
    domain_t_scaled = (scale_t(domain_t[0]), scale_t(domain_t[1]))  # (-1, 1)

    refine_1 = ((scale_x(0.35), scale_x(0.65)), (scale_y(0.35), scale_y(0.65)))
    refine_2 = ((scale_x(0.65), scale_x(2.5)), (scale_y(0.35), scale_y(0.65)))

    # --- Obstacle (scaled) ---
    obstacle_center = (scale_x(0.5), scale_y(0.5))
    obstacle_radius = 0.05
    obstacle_radius_scaled = 2 * obstacle_radius / (domain_x[1] - domain_x[0])
    obstacle_radius2 = obstacle_radius_scaled ** 2

    # --- Region weights ---
    w1, w2 = 0.40, 0.35
    w3 = 1.0 - (w1 + w2)

    timesteps = np.arange(0, total_time + dt, dt)
    chosen_timesteps = np.random.choice(timesteps, size=n_timesteps_per_iter, replace=False)
    chosen_timesteps_scaled = scale_t(chosen_timesteps)

    # --- Vectorized sampling ---
    total_pts = N_per_timestep * n_timesteps_per_iter
    regions = np.random.choice(3, size=total_pts, p=[w1, w2, w3])

    x = np.empty(total_pts, dtype=np.float32)
    y = np.empty(total_pts, dtype=np.float32)
    t = np.repeat(chosen_timesteps_scaled, N_per_timestep).astype(np.float32)

    # Region 1
    mask1 = regions == 0
    n1 = mask1.sum()
    x[mask1] = np.random.uniform(*refine_1[0], n1)
    y[mask1] = np.random.uniform(*refine_1[1], n1)

    # Region 2
    mask2 = regions == 1
    n2 = mask2.sum()
    x[mask2] = np.random.uniform(*refine_2[0], n2)
    y[mask2] = np.random.uniform(*refine_2[1], n2)

    # Region 3 (rest of domain)
    mask3 = regions == 2
    n3 = mask3.sum()
    x[mask3] = np.random.uniform(*domain_x_scaled, n3)
    y[mask3] = np.random.uniform(*domain_y_scaled, n3)

    # --- Remove obstacle points (vectorized) ---
    dx = x - obstacle_center[0]
    dy = y - obstacle_center[1]
    mask_keep = dx**2 + dy**2 >= obstacle_radius2
    x, y, t = x[mask_keep], y[mask_keep], t[mask_keep]

    # --- Fast resampling if not enough points ---
    n_needed = total_pts - len(x)
    if n_needed > 0:
        xr = np.random.uniform(*refine_2[0], n_needed)
        yr = np.random.uniform(*refine_2[1], n_needed)
        tr = np.random.choice(chosen_timesteps_scaled, size=n_needed)
        dxr = xr - obstacle_center[0]
        dyr = yr - obstacle_center[1]
        keep = dxr**2 + dyr**2 >= obstacle_radius2
        x = np.concatenate([x, xr[keep]])
        y = np.concatenate([y, yr[keep]])
        t = np.concatenate([t, tr[keep]])
        x, y, t = x[:total_pts], y[:total_pts], t[:total_pts]

    # Combine and shuffle
    pts_np = np.stack([x, y, t], axis=1)
    np.random.shuffle(pts_np)

    return torch.tensor(pts_np, dtype=torch.float32, device=device)


In [32]:
def sample_inlet(N_per_timestep,
                 seed=None,
                 device="cuda"):

    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)

    # Original domain (before scaling)
    x_in_original = 0.0
    y_bounds_original = (0.0, 1.0)
    t_bounds_original = (0.0, 5.0)
    dt = 0.01

    # --- Scale function from [min, max] → [-1, 1] ---
    def scale(val, vmin, vmax):
        return 2 * (val - vmin) / (vmax - vmin) - 1

    # Scale boundaries
    x_in = scale(x_in_original, 0.0, 2.5)       # if your domain in x is [0, 2.5]
    y_min = scale(y_bounds_original[0], 0.0, 1.0)
    y_max = scale(y_bounds_original[1], 0.0, 1.0)
    t_min = scale(t_bounds_original[0], 0.0, 5.0)
    t_max = scale(t_bounds_original[1], 0.0, 5.0)

    timesteps = np.arange(t_min, t_max + (2 * (dt / (t_bounds_original[1] - t_bounds_original[0]))), 
                          2 * (dt / (t_bounds_original[1] - t_bounds_original[0])))

    all_points = []

    # Generate samples along inlet (x = x_in, y ∈ [-1, 1], t ∈ [-1, 1])
    for t in timesteps:
        y = np.random.uniform(y_min, y_max, N_per_timestep)
        x = np.full(N_per_timestep, x_in)
        t_vals = np.full(N_per_timestep, t)
        pts_np = np.stack([x, y, t_vals], axis=1)
        all_points.append(pts_np)

    all_points = np.vstack(all_points)
    np.random.shuffle(all_points)

    # Convert to torch
    points = torch.tensor(all_points, dtype=torch.float32, device=device)

    return points


In [33]:
def sample_cylinder_surface(N_per_timestep,
                            seed=None,
                            device="cuda"):

    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)

    # Original domain and cylinder info
    total_time = 5.0
    dt = 0.01
    cyl_center = (0.5, 0.5)
    cyl_radius = 0.05

    # Original domain bounds
    x_bounds = (0.0, 2.5)
    y_bounds = (0.0, 1.0)
    t_bounds = (0.0, total_time)

    # --- Scaling function [min, max] → [-1, 1] ---
    def scale(val, vmin, vmax):
        return 2 * (val - vmin) / (vmax - vmin) - 1

    # Scale time range and compute scaled time step
    t_min = scale(t_bounds[0], *t_bounds)
    t_max = scale(t_bounds[1], *t_bounds)
    dt_scaled = 2 * (dt / (t_bounds[1] - t_bounds[0]))  # scaled time step

    timesteps = np.arange(t_min, t_max + dt_scaled, dt_scaled)
    all_points = []

    # Cylinder center and radius scaled to [-1, 1]
    cx_scaled = scale(cyl_center[0], *x_bounds)
    cy_scaled = scale(cyl_center[1], *y_bounds)
    r_scaled_x = 2 * cyl_radius / (x_bounds[1] - x_bounds[0])
    r_scaled_y = 2 * cyl_radius / (y_bounds[1] - y_bounds[0])

    for t in timesteps:
        theta = np.random.uniform(0, 2*np.pi, N_per_timestep)

        # Scale radius independently in x and y directions
        x = cx_scaled + r_scaled_x * np.cos(theta)
        y = cy_scaled + r_scaled_y * np.sin(theta)
        t_vals = np.full(N_per_timestep, t)

        pts_np = np.stack([x, y, t_vals], axis=1)
        all_points.append(pts_np)

    all_points = np.vstack(all_points)
    np.random.shuffle(all_points)

    points = torch.tensor(all_points, dtype=torch.float32, device=device)

    return points


In [34]:
def sample_top_bottom(N_per_timestep,
                      seed=None,
                      device="cpu"):
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)

    # --- Original physical domain ---
    total_time = 5.0
    dt = 0.01
    x_bounds = (0.0, 2.5)
    y_bounds = (0.0, 1.0)
    t_bounds = (0.0, total_time)

    y_top = y_bounds[1]
    y_bot = y_bounds[0]

    # --- Scaling function [min, max] → [-1, 1] ---
    def scale(val, vmin, vmax):
        return 2 * (val - vmin) / (vmax - vmin) - 1

    # --- Pre-scale bounds and constants ---
    x_min_scaled, x_max_scaled = scale(np.array(x_bounds), *x_bounds)
    y_top_scaled = scale(y_top, *y_bounds)
    y_bot_scaled = scale(y_bot, *y_bounds)
    t_min_scaled, t_max_scaled = scale(np.array(t_bounds), *t_bounds)

    # Compute scaled timestep spacing
    dt_scaled = 2 * (dt / (t_bounds[1] - t_bounds[0]))

    timesteps = np.arange(t_min_scaled, t_max_scaled + dt_scaled, dt_scaled)

    # Split evenly between top and bottom
    N_top = N_per_timestep // 2
    N_bot = N_per_timestep - N_top

    all_points = []

    for t in timesteps:
        # --- Top wall ---
        x_top = np.random.uniform(x_min_scaled, x_max_scaled, N_top)
        y_top_arr = np.full(N_top, y_top_scaled)
        t_top = np.full(N_top, t)
        pts_top = np.stack([x_top, y_top_arr, t_top], axis=1)

        # --- Bottom wall ---
        x_bot = np.random.uniform(x_min_scaled, x_max_scaled, N_bot)
        y_bot_arr = np.full(N_bot, y_bot_scaled)
        t_bot = np.full(N_bot, t)
        pts_bot = np.stack([x_bot, y_bot_arr, t_bot], axis=1)

        all_points.append(pts_top)
        all_points.append(pts_bot)

    all_points = np.vstack(all_points)
    np.random.shuffle(all_points)

    points = torch.tensor(all_points, dtype=torch.float32, device=device)
    return points


In [35]:
import torch

def sample_initial(N,
                   xyt_tensor_ic, 
                   uvp_tensor_ic,
                   device="cuda"):

    # --- Original physical bounds ---
    x_bounds = (0.0, 2.5)
    y_bounds = (0.0, 1.0)

    # --- Scaling function ---
    def scale(val, vmin, vmax):
        return 2 * (val - vmin) / (vmax - vmin) - 1

    # --- Scaled geometry and regions ---
    cyl_center = (scale(0.5, *x_bounds), scale(0.5, *y_bounds))
    cyl_radius = 2 * (0.05 / (x_bounds[1] - x_bounds[0]))  # scale length (Δx=2.5)
    
    bound1 = (
        (scale(0.35, *x_bounds), scale(0.65, *x_bounds)),
        (scale(0.35, *y_bounds), scale(0.65, *y_bounds))
    )

    bound2 = (
        (scale(0.65, *x_bounds), scale(2.5, *x_bounds)),
        (scale(0.35, *y_bounds), scale(0.65, *y_bounds))
    )

    # --- Percentages ---
    perc_r1, perc_r2, perc_r3 = 0.4, 0.35, 0.25
    N_r1 = int(N * perc_r1)
    N_r2 = int(N * perc_r2)
    N_r3 = N - N_r1 - N_r2

    # --- Coordinates ---
    x = xyt_tensor_ic[:, 0]
    y = xyt_tensor_ic[:, 1]

    # --- Mask out cylinder ---
    cx, cy = cyl_center
    dx, dy = x - cx, y - cy
    mask_cyl = (dx**2 + dy**2) >= cyl_radius**2
    xyt_valid = xyt_tensor_ic[mask_cyl]
    uvp_valid = uvp_tensor_ic[mask_cyl]

    # --- Region 1 (around cylinder) ---
    mask_r1 = ((xyt_valid[:, 0] >= bound1[0][0]) & (xyt_valid[:, 0] <= bound1[0][1]) &
               (xyt_valid[:, 1] >= bound1[1][0]) & (xyt_valid[:, 1] <= bound1[1][1]))

    # --- Region 2 (wake) ---
    mask_r2 = ((xyt_valid[:, 0] >= bound2[0][0]) & (xyt_valid[:, 0] <= bound2[0][1]) &
               (xyt_valid[:, 1] >= bound2[1][0]) & (xyt_valid[:, 1] <= bound2[1][1]))

    # --- Region 3 = rest ---
    mask_r3 = ~(mask_r1 | mask_r2)

    # --- Sampling helper ---
    def sample_region(mask, N):
        xyt_reg = xyt_valid[mask]
        uvp_reg = uvp_valid[mask]
        if len(xyt_reg) == 0:
            return None, None
        replace = xyt_reg.shape[0] < N
        idx = torch.randint(0, xyt_reg.shape[0], (N,), device="cpu", dtype=torch.long) if replace else \
              torch.randperm(xyt_reg.shape[0])[:N]
        return xyt_reg[idx].to(device), uvp_reg[idx].to(device)

    # --- Sample each region ---
    xyt_r1, uvp_r1 = sample_region(mask_r1, N_r1)
    xyt_r2, uvp_r2 = sample_region(mask_r2, N_r2)
    xyt_r3, uvp_r3 = sample_region(mask_r3, N_r3)

    # --- Concatenate ---
    sampled_xyt = torch.cat([t for t in [xyt_r1, xyt_r2, xyt_r3] if t is not None], dim=0)
    sampled_uvp = torch.cat([t for t in [uvp_r1, uvp_r2, uvp_r3] if t is not None], dim=0)

    return sampled_xyt, sampled_uvp


In [36]:
import torch

def compute_residuals(model, X, sigma_k=1.0, sigma_e=1.3,
                      C1=1.42, C2=1.68, rho=1.0):
    """
    Compute PDE residuals for scaled coordinates (x', y', t') ∈ [-1, 1].
    Handles: continuity, momentum, k, ε equations (RNG k–ε model).
    """

    # --- Physical constants ---
    nu = 5e-4  
    
    # --- Scaling factors (for chain rule) ---
    Lx, Ly, Lt = 2.5, 1.0, 5.0
    sx = 2.0 / Lx   # 0.8
    sy = 2.0 / Ly   # 2.0
    st = 2.0 / Lt   # 0.4

    # --- Forward pass ---
    X = X.clone().detach().requires_grad_(True)
    ψ, p, k, eps = model(X)

    # --- Velocity components from ψ ---
    grads_ψ = torch.autograd.grad(ψ, X, grad_outputs=torch.ones_like(ψ),
                                  create_graph=True, retain_graph=True)[0]
    ψ_xp = grads_ψ[:, 0:1]
    ψ_yp = grads_ψ[:, 1:2]
    # Convert to physical derivatives
    ψ_x = ψ_xp * sx
    ψ_y = ψ_yp * sy

    u = ψ_y
    v = -ψ_x

    # --- Velocity gradients ---
    grads_u = torch.autograd.grad(u, X, grad_outputs=torch.ones_like(u),
                                  create_graph=True, retain_graph=True)[0]
    grads_v = torch.autograd.grad(v, X, grad_outputs=torch.ones_like(v),
                                  create_graph=True, retain_graph=True)[0]

    u_x = grads_u[:, 0:1] * sx
    u_y = grads_u[:, 1:2] * sy
    u_t = grads_u[:, 2:3] * st

    v_x = grads_v[:, 0:1] * sx
    v_y = grads_v[:, 1:2] * sy
    v_t = grads_v[:, 2:3] * st

    # --- Second derivatives ---
    u_xx = torch.autograd.grad(u_x, X, torch.ones_like(u_x), create_graph=True, retain_graph=True)[0][:, 0:1] * sx**2
    u_yy = torch.autograd.grad(u_y, X, torch.ones_like(u_y), create_graph=True, retain_graph=True)[0][:, 1:2] * sy**2
    v_xx = torch.autograd.grad(v_x, X, torch.ones_like(v_x), create_graph=True, retain_graph=True)[0][:, 0:1] * sx**2
    v_yy = torch.autograd.grad(v_y, X, torch.ones_like(v_y), create_graph=True, retain_graph=True)[0][:, 1:2] * sy**2

    # --- Pressure gradients ---
    grads_p = torch.autograd.grad(p, X, grad_outputs=torch.ones_like(p),
                                  create_graph=True, retain_graph=True)[0]
    p_x = grads_p[:, 0:1] * sx
    p_y = grads_p[:, 1:2] * sy

    # --- Continuity ---
    cont = u_x + v_y

    # --- Turbulent viscosity ---
    nu_t = k**2 / (eps + 1e-8)
    nu_eff = nu + nu_t

    # --- Momentum residuals ---
    mom_u = u_t + (u * u_x + v * u_y) + (1 / rho) * p_x - nu_eff * (u_xx + u_yy)
    mom_v = v_t + (u * v_x + v * v_y) + (1 / rho) * p_y - nu_eff * (v_xx + v_yy)

    # --- k-equation residual ---
    grads_k = torch.autograd.grad(k, X, grad_outputs=torch.ones_like(k),
                                  create_graph=True, retain_graph=True)[0]
    k_x = grads_k[:, 0:1] * sx
    k_y = grads_k[:, 1:2] * sy
    k_t = grads_k[:, 2:3] * st
    k_xx = torch.autograd.grad(k_x, X, torch.ones_like(k_x), create_graph=True, retain_graph=True)[0][:, 0:1] * sx**2
    k_yy = torch.autograd.grad(k_y, X, torch.ones_like(k_y), create_graph=True, retain_graph=True)[0][:, 1:2] * sy**2

    # Production term Pk
    Sxx = u_x
    Syy = v_y
    Sxy = 0.5 * (u_y + v_x)
    Pk = 2 * nu_t * (Sxx**2 + 2*Sxy**2 + Syy**2)

    res_k = (
        k_t
        + (u * k_x + v * k_y)
        - Pk
        + eps
        - (nu + nu_t / sigma_k) * (k_xx + k_yy)
    )

    # --- ε-equation residual ---
    grads_eps = torch.autograd.grad(eps, X, grad_outputs=torch.ones_like(eps),
                                    create_graph=True, retain_graph=True)[0]
    e_x = grads_eps[:, 0:1] * sx
    e_y = grads_eps[:, 1:2] * sy
    e_t = grads_eps[:, 2:3] * st
    e_xx = torch.autograd.grad(e_x, X, torch.ones_like(e_x), create_graph=True, retain_graph=True)[0][:, 0:1] * sx**2
    e_yy = torch.autograd.grad(e_y, X, torch.ones_like(e_y), create_graph=True, retain_graph=True)[0][:, 1:2] * sy**2

    R = torch.zeros_like(k)  # RNG correction term placeholder
    res_eps = (
        e_t
        + (u * e_x + v * e_y)
        - (C1 - R) * (eps / k) * Pk
        + C2 * eps**2 / k
        - (nu + nu_t / sigma_e) * (e_xx + e_yy)
    )

    return cont, mom_u, mom_v, res_k, res_eps


In [37]:
def data_loss(model, pts, u, v, p, device):

    # Model prediction
    u_pred, v_pred, p_pred = model.velocity_pressure(pts)

    mse = nn.MSELoss()
    return mse(u_pred, u) + mse(v_pred, v) + mse(p_pred, p)


In [38]:
os.makedirs('models', exist_ok=True)
Nf, Nf_data, Nic, Nd, Nn, Ni = 40000, 10000, 50000, 60, 700, 150

In [39]:
def compute_residuals_in_batches(model, X, batch_size=10000):

    cont_list, mu_list, mv_list, rk_list, re_list = [], [], [], [], []
    n = X.size(0)

    for i in range(0, n, batch_size):
        X_batch = X[i:i+batch_size].clone().detach().requires_grad_(True)
        
        # Compute all residuals for this batch
        cont, mom_u, mom_v, res_k, res_eps = compute_residuals(model, X_batch)

        # Store each batch result
        cont_list.append(cont.detach())
        mu_list.append(mom_u.detach())
        mv_list.append(mom_v.detach())
        rk_list.append(res_k.detach())
        re_list.append(res_eps.detach())

        # Free up memory
        del cont, mom_u, mom_v, res_k, res_eps, X_batch
        torch.cuda.empty_cache()

    # Concatenate results back into full tensors
    return (
        torch.cat(cont_list, dim=0),
        torch.cat(mu_list, dim=0),
        torch.cat(mv_list, dim=0),
        torch.cat(rk_list, dim=0),
        torch.cat(re_list, dim=0)
    )

In [40]:
import torch
import torch.optim as optim
import torch.nn as nn
import os
from torch.cuda.amp import autocast, GradScaler


mse_loss = nn.MSELoss()

def train_adam(model, Nf, Nf_data, Nic, Nd, Nn, Ni,
               num_iters=40000, print_every=2000, save_every=2000,
               λ_data=1.0, alpha_pde=1.0, seed=None, device=None,
               checkpoint_dir="models"):
    
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training on device: {device}")
    print("Start training loop")

    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer,
        T_0=100,      # restart every 100 iterations
        T_mult=1,     # keep restart period constant
        eta_min=1e-6  # minimum LR between restarts
    )
    scaler = GradScaler()

    start_iter = 0

    # --- Try to resume from the latest checkpoint ---
    latest_ckpt = None
    if os.path.exists(checkpoint_dir):
        ckpts = [f for f in os.listdir(checkpoint_dir) if f.startswith("pinn_checkpoint_")]
        if ckpts:
            latest_ckpt = max(ckpts, key=lambda f: int(f.split("_")[-1].split(".")[0]))

    if latest_ckpt:
        path = os.path.join(checkpoint_dir, latest_ckpt)
        print(f"Loading checkpoint from {path}...")
        checkpoint = torch.load(path, map_location=device)
        model.load_state_dict(checkpoint["model_state"])
        optimizer.load_state_dict(checkpoint["optimizer_state"])
        start_iter = checkpoint["iter"] + 1
        print(f"Resuming training from iteration {start_iter}")


    for it in range(start_iter, num_iters):
        optimizer.zero_grad()

        with autocast():

            # Physics residual loss
            X_f = sample_collocation_points(N_per_timestep=Nf,
                                              seed=None,
                                              device="cuda")
            
            X_f_data, u, v, p = sample_points(inputs=xyt_tensor,
                                              outputs=uvp_tensor,
                                              N_per_timestep=Nf_data,
                                              seed=None,
                                              device="cuda")
        
    
    
                        # Compute residuals in batches
            cont, mom_u, mom_v, res_k, res_e = compute_residuals_in_batches(model, X_f, batch_size=10000)
            
            # PDE loss: continuity, momentum (u,v), turbulence (k, ε)
            loss_f = (
                mse_loss(cont, torch.zeros_like(cont)) +
                mse_loss(mom_u, torch.zeros_like(mom_u)) +
                mse_loss(mom_v, torch.zeros_like(mom_v)) +
                mse_loss(res_k, torch.zeros_like(res_k)) +
                mse_loss(res_e, torch.zeros_like(res_e))
            )
    
    
            # Data loss
            loss_data = data_loss(model, X_f_data, u, v, p, device="cuda")
    
    
            # Initial condition
            X_ic, Y_ic = sample_initial(N=Nic,
                                        xyt_tensor_ic=xyt_tensor_ic,
                                        uvp_tensor_ic=uvp_tensor_ic, 
                                        device="cuda")
    
            
            u_ic, v_ic, p_ic = model.velocity_pressure(X_ic)
            loss_ic = (mse_loss(u_ic, Y_ic[:, 0:1]) + 
                       mse_loss(v_ic, Y_ic[:, 1:2]) + 
                       mse_loss(p_ic, Y_ic[:, 2:3]))
    
    
            # Inlet condition
            X_in = sample_inlet(N_per_timestep=Ni,
                                seed=None, 
                                device="cuda")
    
            
            u_in, v_in, p_in = model.velocity_pressure(X_in)
            loss_in = (mse_loss(u_in, torch.ones_like(u_in)) + 
                       mse_loss(v_in, torch.zeros_like(v_in))
                      )
    
    
            # Cylinder surface
            X_cyl = sample_cylinder_surface(N_per_timestep=Nd,
                                            seed=None, 
                                            device="cuda")
    
            
            u_cyl, v_cyl, p_cyl = model.velocity_pressure(X_cyl)

            loss_cyl = (
                mse_loss(u_cyl, torch.zeros_like(u_cyl)) +  # u = 0
                mse_loss(v_cyl, torch.zeros_like(v_cyl))    # v = 0
            )
        
    
            # Top/bottom boundaries
            X_tb = sample_top_bottom(N_per_timestep=Nn, 
                                     seed=None, 
                                     device="cuda")
    
            u_tb, v_tb, p_tb = model.velocity_pressure(X_tb)

            loss_tb = (
                mse_loss(v_tb, torch.zeros_like(v_tb))      # v = 0 (no vertical velocity)
            )
                
    
            # Total loss
            loss = alpha_pde * loss_f + λ_data * loss_data + loss_ic + loss_in + loss_cyl + loss_tb


        
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step(it + 1)

        torch.cuda.empty_cache()


        if it % print_every == 0:
            current_lr = optimizer.param_groups[0]["lr"]
            print(f"Adam Iter {it}/{num_iters} - LR {current_lr:.2e} - Total loss {loss.item():.6f}\n"
                  f"Data {loss_data.item():.6f} | PDE {loss_f.item():.4e}\n"
                  f"IC {loss_ic.item():.4e}, Inlet {loss_in.item():.4e}\n"
                  f"Cylinder {loss_cyl.item():.4e}, TopBottom {loss_tb.item():.4e}")

         # --- Save checkpoint every save_every iterations ---
        if (it + 1) % save_every == 0:
            os.makedirs(checkpoint_dir, exist_ok=True)
            ckpt_path = os.path.join(checkpoint_dir, f"resnet_pinn_checkpoint_{it+1}.pth")
            torch.save({
                "iter": it,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "scaler_state": scaler.state_dict()
            }, ckpt_path)
            print(f"Checkpoint saved → {ckpt_path}")

         
            
    current_lr = optimizer.param_groups[0]["lr"]
    print(f"Adam Iter {num_iters-1} - LR {current_lr:.2e} - Total loss {loss.item():.6f}\n"
              f"Data {loss_data.item():.6f} | PDE {loss_f.item():.4e}\n"
              f"IC {loss_ic.item():.4e}, Inlet {loss_in.item():.4e}\n"
              f"Cylinder {loss_cyl.item():.4e}, TopBottom {loss_tb.item():.4e}")


    # Final model save
    os.makedirs(checkpoint_dir, exist_ok=True)
    final_path = os.path.join(checkpoint_dir, "resnet_pinn_ns_adam_final.pth")
    torch.save(model.state_dict(), final_path)
    print(f"Final model saved at {final_path}")


In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PINN().to(device)

In [ ]:
train_adam(model, Nf, Nf_data, Nic, Nd, Nn, Ni,
               num_iters=1000, print_every=10, save_every=100,
               λ_data=1.0, alpha_pde=1e-3, seed=None, device=device,
               checkpoint_dir="models")

Training on device: cuda
Start training loop


/tmp/ipykernel_39/2235816808.py:27: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_39/2235816808.py:51: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Adam Iter 0/1000 - LR 1.00e-04 - Total loss 1.868529
Data 0.526898 | PDE 2.2048e+02
IC 2.2028e-01, Inlet 1.3527e-01
Cylinder 7.5164e-01, TopBottom 1.3969e-02
Adam Iter 10/1000 - LR 9.71e-05 - Total loss 1.439418
Data 0.341254 | PDE 1.2499e+02
IC 2.1553e-01, Inlet 1.6030e-01
Cylinder 5.8882e-01, TopBottom 8.5301e-03
Adam Iter 20/1000 - LR 8.96e-05 - Total loss 1.262884
Data 0.256053 | PDE 9.7293e+01
IC 1.9793e-01, Inlet 1.6104e-01
Cylinder 5.4468e-01, TopBottom 5.8786e-03
Adam Iter 30/1000 - LR 7.83e-05 - Total loss 1.091079
Data 0.159499 | PDE 7.0977e+01
IC 1.8630e-01, Inlet 1.5932e-01
Cylinder 5.1004e-01, TopBottom 4.9446e-03
Adam Iter 40/1000 - LR 6.43e-05 - Total loss 1.016320
Data 0.125907 | PDE 7.2354e+01
IC 1.8008e-01, Inlet 1.5795e-01
Cylinder 4.7457e-01, TopBottom 5.4646e-03
Adam Iter 50/1000 - LR 4.89e-05 - Total loss 0.945138
Data 0.108874 | PDE 5.1142e+01
IC 1.7744e-01, Inlet 1.5855e-01
Cylinder 4.4179e-01, TopBottom 7.3503e-03
Adam Iter 60/1000 - LR 3.37e-05 - Total loss 0.